# 실습 목표
- 추천시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
- 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

# 목차
- 추천시스템이란
- 데이터 탐색하기와 전처리
- 사용자의 명시적/암묵적 평가
- Matrix Factorization(MF)
- CSR(Compressed Sparse Row) Matrix
- MF 모델 학습하기
- 비슷한 아티스트 찾기 + 유저에게 추천하기
- 프로젝트 - MovieLens 영화 추천 실습

# 추천시스템이란?
[콘텐츠 추천 알고리즘의 진화](http://www.kocca.kr/insight/vol05/vol05_04.pdf)

Q1. 협업 필터링(Collaborative Filtering) 방식과 콘텐츠 기반 필터링(Contents-based Filtering) 방식의 **차이점**이 무엇인가요?
> **협업 필터링**은 다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악하지만,
**콘텐츠 기반 필터링**은 아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악합니다.
다시 말해, 협업 필터링에서는 **아이템과 사용자 간의 행동 또는 관계에만** 주목할 뿐 아이템 자체의 고유한 속성에 주목하지 않습니다.
콘텐츠 기반 필터링에서는 **아이템 자체의 속성에만** 주목하고 사용자와 아이템 간의 관련성 에는 주목하지 않습니다.

Q2. 다수의 사용자의 판단을 기반으로 정확한 추천을 가능하게 하는 추천시스템의 핵심 근간은 협업 필터링입니다만, **협업 필터링**을 바로 사용할 수 없게 만드는 세가지 **제약 조건**은 무엇일까요?
> 1. 시스템이 충분한 정보를 모으지 못한 사용자나 아이템에 대한 추론을 할 수 없는 상태인 콜드 스타트(Cold Start) 상황
2. 계산량이 너무 많아 추천의 효율이 떨어지는 상황
3. 롱테일의 꼬리 부분, 즉 사용자의 관심이 저조한 항목의 정보가 부족하여 추천에서 배제되는 상황

Q3. 유튜브 뮤직의 첫화면에서 처음 접속한 사용자에게 좋아하는 아티스트 5명 이상의 정보를 요구하는 이유는 무엇일까요? (생각해 보세요)
> 처음 접속한 사용자라면 이 사용자에 대한 아무런 초기 정보가 없는 상황이어서 사용자 기반 추천을 전혀 할 수 없어 콘텐츠 기반 필터링 방식의 추천만 가능한데, 이를 보완하여 처음부터 사용자 특성에 따른 맞춤형 서비스를 제공하기 위해서입니다.
사용자 맞춤형 추천을 제공하지 못한다면 모든 초기 사용자에게 같은 화면을 제시할 수밖에 없는데, 이러면 모든 사용자들이 처음 추천받은 콘텐츠 위주로만 시청하게 되어 추천시스템이 빠지기 쉬운 필터 버블의 문제을 야기하거나, 추천의 다양성을 저해할 수 있는 위험이 생깁니다.

추천시스템은 이처럼
**아이템은 매우 많고, 
유저의 취향은 다양할 때**
유저가 소비할만한 아이템을 예측하는 모델입니다.

<br>

오늘 우리는 이렇게 쌓인 데이터 사례 중 하나를 활용하여 유저가 좋아하는 특정 아티스트와 유사한 다른 아티스트를 추천하는 추천시스템을 만들어 보려고 합니다. Last.fm에서는 어떤 유저가 특정 아티스트의 노래를 몇 번이나 들었는지에 대한 데이터를 제공하고 있습니다. 2010년에 미국에서 서비스되고 있는 spotify에서 발생한 데이터이기 때문에 생소한 아티스트가 많을 수 있습니다.

# 데이터 탐색하기와 전처리
tsv는 Tab-Separated Values로, tap문자가 구분자인 데이터파일이다.<br>

[데이터셋 홈페이지](tsv는 Tab-Separated Values)에서 항목정의가 ( user-mboxsha1 \t musicbrainz-artist-id \t artist-name \t plays ) 라고 되어 있는 것으로 보아, 4개의 항목은 각각 User ID, Artist MBID, Artist Name, Play횟수 로 보입니다.

### 데이터 준비

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


artist_MBID 항목은 우리의 관심사가 아닙니다. 사용하는 컬럼만 남기고 제거하도록 하겠습니다.

In [2]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔 줍시다.

In [3]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해 봅시다.

In [4]:
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


### 데이터 탐색
추천 모델을 만들기 전에 데이터의 기본적인 정보를 보기 위해 아래 항목들을 확인해보자.

- 유저수, 아티스트수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

(참고) **pandas.DataFrame.nunique()**은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용하다.

In [5]:
# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [11]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

### 모델 검증을 위한 사용자 초기 정보 세팅
우리는 본인의 음악 취향과 가장 유사한 아티스트를 추천받고 싶습니다. 유튜브 뮤직 등 추천시스템들은 이를 위해서 처음 가입하는 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많습니다.

우리도 동일한 과정이 필요합니다. 우리가 만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 저는 아래와 같이 제가 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣겠습니다.

In [12]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_playlist 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


### 모델에 활용하기 위한 전처리
사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 **user와 artist 각각에 번호를 붙이고 싶습니다.** 보통 이런 작업을 **indexing**이라고 합니다. 추천시스템, 자연어처리에서 자주 하는 작업들이라 직접 해보시면 좋겠습니다.

(참고) **pandas.DataFrame.unique()**은 특정 컬럼에 포함된 유니크한 데이터만 **리스트로 모아 줍니다.** indexing 작업을 위해 매우 유용합니다.

In [13]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [15]:
user_unique

array(['00000c289a1829a808ac09c00daf10bc3c4e223b',
       '00001411dc427966b17297bf4d69e7e193135d89',
       '00004d2ac9316e22dc007ab2243d6fcb239e707d', ...,
       'ffff9ef87a7d9494ada2f9ade4b9ff637c0759ac', 'sep 20, 2008',
       'zimin'], dtype=object)

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


**Series.map()** 함수는Series의 값을 대체합니다. 대체 값은 Series, 딕셔너리 또는 함수에서 파생 될 수 있습니다. <br>
[참고](https://www.delftstack.com/ko/api/python-pandas/pandas-series-series.map-function/)

In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# map함수로 딕셔너리user_to_idx를 통해 user_id 컬럼의 모든 값을 인덱싱한 값으로 대체한다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거한다. 
temp_user_data = data['user_id'].map(user_to_idx).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


# 사용자의 명시적/암묵적 평가

[Explicit vs. Implicit Feedback Datasets](https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301)

Q. 암묵적 평가가 될 수 있는 데이터의 사례에는 어떤 것이 있을지 생각해 봅시다.
> 플레이 횟수, 플레이 시간 이외에도 클릭 수, 구매 여부, 플레이 스킵 여부, 검색 기록, 방문 페이지 이력, 구매 내역, 심지어 마우스 움직임 기록도 암묵적 평가 데이터셋이 될 수 있습니다.

Q. 어떤 유저가 아티스트의 곡을 한 번만 들었다는 것의 의미를 생각해 봅시다. 유저는 이 아티스트를 좋아하는 걸까요 싫어하는 걸까요? 이 유저에게 이 아티스트와 관련된 사람을 추천해야 할까요?
> 이것은 명확한 정답은 존재하지 않습니다. 어떤 유저는 한 번 듣고 별로라고 생각했을 수도 있고, 어떤 유저는 다시 듣고 싶지만 가수의 이름을 잊어버렸을 수도 있습니다. 이런 애매한 암묵적 데이터야말로 도메인 지식과 직관이 활용되어야 하는 영역입니다.

그렇다면 우리의 데이터셋에서 1회만 플레이한 데이터의 비율은 어떻게 될까?

In [19]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']==1]
one, all_data = len(only_one), len(data)

print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147739,17535660
Ratio of only_one over all data is 0.84%


우리가 앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것입니다.

- 한 번이라도 들었으면 선호한다고 판단한다.
- 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

# Matrix Factorization (MF, 행렬분해)

데이터가 준비되었으니 이제 모델을 만들 차례입니다!

아래 그림 왼쪽의 행렬처럼, 우리는 **m명의 사용자**들이 **n명의 아티스트에 대해 평가**한 데이터를 포함한 (m,n) 사이즈의 **평가행렬(Rating Matrix)**을 만들 것입니다. 행렬 중 일부는 데이터가 채워져 있지만, 나머지 부분은 데이터가 비어 있습니다. 우리가 앞에서 살펴보았던 추천시스템의 협업 필터링(Collaborative Filtering)이란 결국은 이런 평가행렬을 전제로 하는 것입니다. 만약 우리가 이 평가행렬의 비어 있는 부분을 포함한 완벽한 정보를 얻을 수만 있다면 (모든 사용자의 모든 아이템에 대한 선호도를 다 파악한 셈이니) 그야말로 완벽한 추천이 가능해질 것입니다.

우리는 추천 시스템의 다양한 모델 중 **Matrix Factorization(MF, 행렬분해) 모델을 사용**하겠습니다. MF 모델은 2006년 Netflix에서 백만달러의 상금을 걸고 개최한 자사 추천시스템의 성능을 10% 이상 향상시키는 챌린지를 계기로 알려지게 되었습니다. 기본 아이디어는 아래 그림과 같습니다. **(m,n) 사이즈의 평가행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해**한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어입니다. 대체로 k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리해집니다. 하지만 아이디어의 단순함에도 불구하고 **MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델입니다.**

![이미지](https://drive.google.com/uc?export=view&id=1tQuWTg1iRkqOWYv_Gexc9XJsJy91nY1U)

(출처: CuMF_SGD: Fast and Scalable Matrix Factorization(https://arxiv.org/pdf/1610.05838.pdf))


<br>

아래 그림을 위 그림방식으로 표현하면 m=4(사용자수), n=5(아이템수), k=2인 MF모델이다.

MF모델은 큰 평가행렬R을 두 개의 Feature Matrix P와 Q로 분해한다.
- 행렬 R: m x n = 4 x 5
- 행렬 P: m x k = 4 x 2
- 행렬 Q: k x n = 2 x 5

그럼 P와 Q에서 무엇이 feature일까?
- 행렬 P의 첫 번째 벡터 $P_0=(1, 0.1)$은 첫 번째 사용자의 특성 벡터(feature vector)가 된다.
- 행렬 Q의 첫 번째 벡터 $Q_0=(0.9, -0.2)$는 첫 번째 아이템인 해리포터 영화의 특성 벡터가 된다.

<br>

이 두 벡터를 내적해서 얻어지는 $R_{0,0}=0.88$이 바로 사용자0의 영화0에 대한 선호도로 보는 것이다.

![이미지](https://drive.google.com/uc?export=view&id=1YJUYwXDsvyKLDyCJLNjKhz5dcCNwjpqP)
(출처: https://developers.google.com/machine-learning/recommendation/collaborative/matrix)

**모델의 목표**는 모든 유저와 아이템에 대해 **k-dimension의 벡터를 잘 만드는 것**입니다.<br>
벡터를 잘 만드는 **기준**은 **유저i의 벡터($U_i$)와 아이템j의 벡터($I_j$)를 내적했을 때 유저i가 아이템j에 대해 평가한 수치($M_{ij}$)와 비슷한지** 입니다.<br>
(유저의 벡터는 행렬 P, 아이템의 벡터는 행렬 Q에서 구할 수 있다.)

MF에도 다양한 변형이 있습니다. 이번에 사용할 모델은 [Collaborative Filtering for Implicit Feedback Datasets 논문](http://yifanhu.net/PUB/cf.pdf)에서 제안한 모델을 사용할 것입니다.

이 모델의 경우 이전 스텝에서 제가 'black eyed peas'를 play했다는 데이터를 추가했기 때문에 'zimin'의 벡터와 black eyed peas의 벡터를 곱했을 때 1에 가까워야 모델이 잘 학습하는 것을 목표로 합니다.

[참고자료) Recommender System — Matrix Factorization](https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b)

# CSR(Compressed Sparse Row) Matrix

유저 X 아이템 평가행렬을 다시 생각해 봅시다. 유저는 36만 명이고 아티스트는 29만 명입니다. 이를 행렬로 표현하고 행렬의 각 원소에 정수 한 개 (1byte)가 들어간다면 36만\*29만\*1byte $\approx$ 97GB가 필요합니다.

Q. 평가행렬 용량이 이렇게 큰 이유는 무엇일까요? 메모리 공간을 낭비하게 되는 요소는 무엇일까요?
> 유저 수 X 아티스트 수만큼의 정보 안에는 유저가 들어보지 않은 아티스트에 대한 정보까지 모두 행렬에 포함되어 계산되기 때문입니다. 아무리 많은 아티스트를 들어 본 유저도 수백 명을 넘어갈 만큼의 아티스트를 들어보지 못했는데, 총 아티스트는 29만 명이 넘기 때문에 평가행렬 내의 대부분의 공간은 0으로 채워집니다. 이런 행렬을 Sparse Matrix라고 합니다. 이런 메모리 낭비를 최소화하기 위해서는 유저가 들어본 아티스트에 대해서만 정보만을 저장하면서 전체 행렬 형태를 유추할 수 있는 데이터 구조가 필요합니다.


참고로 여러분들이 사용하고 계실 컴퓨터의 메모리는 많아야 16GB일 테니 97GB나 되는 거대한 행렬을 메모리에 올려놓고 작업한다는 것은 불가능할 것입니다. 이런 경우의 좋은 대안이 되는 것이 **CSR(Compressed Sparse Row) Matrix**입니다. **모델 학습의 input으로 사용할 데이터 타입**을 CSR Matrix로 할 것입니다.

CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다.<br>
[참고1) Scipy sparse matrix handling](https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix)<br>
[참고2) StackOverflow csr_matrix 설명](https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005)
[참고3) scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)
- csr_matrix를 만드는 방법은 다양하게 있습니다.
- 현재 data와 같은 데이터 구조에 적합한 방식은 위 링크에서 4번째로 설명하고 있는 방법입니다.


In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Row format>

# MF 모델 학습하기
이전 스텝에서 설명한 Matrix Factorization 모델을 [implicit](https://github.com/benfred/implicit) 패키지를 사용하여 학습해 봅시다.
- implicit 패키지는 이전 스텝에서 설명한 **암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습**할 수 있는 패키지입니다.
- 이 패키지에 구현된 **als(AlternatingLeastSquares) 모델을 사용**하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, **한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식**이 효과적인 것으로 알려져 있습니다.


In [21]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 \_\_init__ 파라미터
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

파라미터 factors와 iterations를 늘릴수록 학습데이터를 잘 학습하게 되지만 overfitting의 우려가 있으니 좋은 값을 찾아야 합니다.

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델 학습이 끝났습니다! 이제 2가지를 아래 사항을 살펴보도록 하겠습니다.

- Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [25]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [26]:
zimin_vector

array([-6.73505962e-02, -1.59925401e-01, -5.57871521e-01, -2.33054504e-01,
       -4.06346530e-01, -3.96142989e-01, -7.16743842e-02, -3.76309991e-01,
        9.20008779e-01,  1.47391498e+00, -4.08733249e-01,  1.28035828e-01,
       -1.30945712e-01, -8.29827487e-02, -6.50555551e-01,  5.50442040e-01,
       -4.01999615e-02,  1.30246294e+00,  4.25499946e-01, -9.20114696e-01,
        2.13653758e-01, -4.40166235e-01,  4.54566926e-01,  8.15658271e-02,
        7.47412264e-01, -9.03512478e-01,  7.98832953e-01,  3.20855826e-02,
        2.01336697e-01, -7.73373306e-01, -4.75167125e-01,  3.77615402e-03,
       -5.40122688e-01,  6.83180571e-01, -3.91552271e-03,  1.32415695e-02,
       -1.61140651e-01,  1.93371251e-01,  8.91608119e-01,  6.06733680e-01,
        6.57297432e-01, -1.33900612e-04, -1.91257671e-01,  3.15233886e-01,
       -1.09422827e+00,  7.97657430e-01,  7.53073275e-01,  1.53532374e+00,
       -2.65984893e-01, -1.63383222e+00,  3.74453843e-01, -1.71822473e-01,
       -1.75970626e+00,  

In [27]:
black_eyed_peas_vector

array([ 0.00253272,  0.00181324,  0.00206538, -0.0045632 ,  0.00702442,
        0.00240971,  0.00300717,  0.00362905,  0.00185747,  0.02661883,
        0.01717748,  0.00068686,  0.00400154, -0.00204526, -0.00930893,
        0.01525597, -0.00465181,  0.03510742,  0.02442543, -0.00993585,
       -0.00240269, -0.00179933,  0.00291981,  0.00745329,  0.02424204,
       -0.01097951,  0.01009851,  0.01473565,  0.01985456, -0.00031859,
        0.02295963,  0.00724326,  0.0035452 ,  0.0233894 , -0.00091305,
       -0.00115105,  0.01440812,  0.0028045 ,  0.00646583,  0.01307095,
        0.01312705,  0.01273685,  0.00917115,  0.00854727, -0.00737314,
        0.02490553,  0.02195368,  0.02674144,  0.01416396, -0.0183192 ,
        0.02242084, -0.00792434, -0.01399266,  0.00741075,  0.00164423,
        0.02369934,  0.00419123,  0.00310894, -0.00274959, -0.0008218 ,
       -0.00603857, -0.00085741,  0.01276397,  0.02221463,  0.00909865,
        0.01075432,  0.00806221, -0.01008223,  0.0004697 ,  0.00

In [28]:
# zimin과 black_eyed_peas를 내적
np.dot(zimin_vector, black_eyed_peas_vector)

0.5094881

1이 나와야 하는데 한참 낮은 수치인 0.50정도가 나왔습니다. factors를 늘리거나 iterations를 늘려야 할 것 같습니다. 이후 진행되는 내용은 일단 이렇게 학습된 모델을 사용하겠습니다.

그렇다면 모델이 'zimin'의 queen에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다

In [29]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.31811243

Q. queen에 대한 선호도를 예측한 결과를 어떻게 해석하면 좋을까요? 사용자와 아티스트 벡터 내적이 0.5를 기준으로 그보다 높으면 모델이 선호한다고 판단했다고 말할 수 있을까요?
> 모델을 만드는 사람이 정하기 나름입니다. 객관적인 지표로 만들어서 기준을 정할 수도 있고 도메인 경험을 통해 정할 수도 있습니다.
추천 시스템은 다른 머신러닝 task보다 객관적인 평가가 어려운 분야입니다. 사람을 도통 알 수 없기 때문에 객관적인 지표를 정하기 어렵기 때문입니다.

# 비슷한 아티스트 찾기 + 유저에게 추천하기

## 비슷한 아티스트 찾기
AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾습니다.

처음으로는 제가 좋아하는 coldplay로 찾아보겠습니다.

In [30]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)

similar_artist

[(62, 0.99999994),
 (277, 0.9871914),
 (28, 0.9799721),
 (5, 0.97741526),
 (217, 0.9773944),
 (473, 0.9720992),
 (247, 0.97074944),
 (490, 0.96766585),
 (418, 0.95764095),
 (268, 0.95482683),
 (910, 0.9492258),
 (1018, 0.9427419),
 (694, 0.94249576),
 (55, 0.9416196),
 (782, 0.9406365)]

(아티스트의 id, 유사도) Tuple 로 반환하고 있습니다. 아티스트의 id를 다시 아티스트의 이름으로 매핑 시켜 주겠습니다.

In [31]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'placebo',
 'the beatles',
 'oasis',
 'u2',
 'pink floyd',
 'nirvana',
 'the smashing pumpkins',
 'foo fighters',
 'arctic monkeys',
 'the white stripes']

몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보겠습니다.

In [32]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

In [33]:
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'dr. dre',
 'the game',
 'nas',
 'snoop dogg',
 'jay-z',
 'bone thugs-n-harmony',
 '50 cent',
 'ludacris']

우와! 스눕독, 닥터 드레, 노토리우스, 나스, 제이지 같은 전설들이 추천됩니다. 힙합의 경우 마니아들이 많아서 더 잘 추천되는 것으로 생각됩니다.


Q. 마니아는 데이터상에서 표현될 때 어떤 특징을 가질까요?
> 마니아들은 특정 장르의 아티스트들에게로 선호도가 집중되고, 다른 장르의 아티스트들과는 선호도가 낮게 나타날 것입니다. 이런 마니아들의 존재로 인해 같은 장르의 아티스트들의 벡터들도 더 가까워져서 get_similar_artist시 장르별 특성이 두드러지게 될 것입니다.

In [34]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'leona lewis',
 'kelly clarkson',
 'justin timberlake']

lady gaga의 경우 여자 아티스트들이 추천되고 있습니다.

### 유저에게 아티스트 추천하기
AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [35]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.45018545),
 (369, 0.43245918),
 (550, 0.43146852),
 (354, 0.42186826),
 (409, 0.40551353),
 (355, 0.4047466),
 (621, 0.40245375),
 (1800, 0.39813885),
 (2249, 0.3949021),
 (627, 0.39245045),
 (274, 0.3780687),
 (391, 0.37578064),
 (5555, 0.3739863),
 (724, 0.3708192),
 (564, 0.3668415),
 (901, 0.36505985),
 (2902, 0.36093634),
 (637, 0.35433343),
 (618, 0.34850812),
 (24, 0.34792358)]

In [36]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'justin timberlake',
 'britney spears',
 'nelly furtado',
 'amy winehouse',
 'madonna',
 'alicia keys',
 'lady gaga',
 'katy perry',
 'maroon 5',
 'michael jackson',
 'christina aguilera',
 'timbaland',
 'lily allen',
 'kanye west',
 'pink',
 'james blunt',
 'akon',
 'the pussycat dolls',
 'jack johnson']

rihanna를 추천해주고 있네요. 모델은 왜 rihanna를 추천해줬을까요? AlternatingLeastSquares 클래스에 구현된 **explain 메소드**를 사용하면 제가 기록을 남긴 데이터 중 이 **추천에 기여한 정도**를 확인할 수 있습니다.

In [37]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [38]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.23096621168549156),
 ('black eyed peas', 0.1339009697574515),
 ('jason mraz', 0.046664431669717475),
 ('coldplay', 0.04041561908362929),
 ('maroon5', 9.445477821217762e-05)]

beyoncé와 black eyed peas가 가장 크게 기여했네요. 둘 다 흑인 아티스트이고 beyoncé의 경우는 같은 성별이라는 점도 크게 작용하지 않았을까 싶습니다.

> 눈치채신 분도 있겠지만 결과가 조금 이상합니다. filter_already_liked_items=True로 했는데 앞에 maroon 5는 제가 이미 평가한 아이템이 나오고 있습니다. 이것은 implicit 버전 0.4.2에서 생긴 버그입니다. 

### 우리가 만든 모델의 아쉬운 점
1. 유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않습니다. 연령대별로 음악 취향이 굉장히 다르지 않을까요?
2. 유저가 언제 play 했는지에 반영하기 쉽지 않습니다. 10년 전에 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트를 비교해보세요.

이러한 이유와 딥러닝의 발전으로 MF 이외의 모델 구조도 많이 연구/ 사용되고 있습니다. 하지만 어떤 추천 모델도 핵심은 MF와 비슷합니다.
유저와 아이템에 대한 벡터를 잘 학습하여 취향에 맞게(유저에 맞게) 아이템을 보여주거나(Retrieval) 걸러내는(Filtering) 역할입니다.